<a href="https://colab.research.google.com/github/simranmasand/CS226-Project-Fall22/blob/main/CSM226_Project_Playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
#from sklearn.linear_model import LinearRegression
#from sklearn.linear_model import LogisticRegression

#pd.set_option('display.max_columns', None)

## DATA IMPORTING, CLEANING, PREPROCESSING

In [160]:
pheno = pd.read_csv("animal/mouse.pheno", delimiter=",")
pheno = pheno[pheno['bw_8wks'].notna()]
pheno = pheno[pheno['bw_presac'].notna()]

In [161]:
pheno

,strain,sex,id,diet,bw_8wks,bw_9wks,bw_10wks,bw_11wks,bw_12wks,bw_13wks,...,hct,rbc,epid_fat_wt,muscle_wt,heart_wt,kidney_wt,liver_wt,perirenal_wt,subcut_adip_wt,soleus_wt
0,BXD1,m,001-0521,CD,22.03,23.06,24.00,23.51,24.45,24.59,...,NaN,NaN,0.2563,0.2232,0.1304,0.4561,1.0010,0.1757,0.0550,0.0100
1,BXD1,m,001-0522,CD,23.78,24.74,25.48,25.59,25.89,26.99,...,NaN,NaN,0.3224,0.2669,0.1561,0.5188,1.1503,0.3038,0.2268,0.0128
2,BXD1,m,001-0523,CD,23.73,24.65,25.80,25.79,26.33,27.30,...,NaN,NaN,0.3810,0.2408,0.1486,0.4614,0.9903,0.2972,0.1621,0.0105
3,BXD1,m,001-0524,CD,25.04,25.75,26.83,27.67,28.41,28.76,...,NaN,NaN,1.0905,0.2766,0.1512,0.5011,1.1661,0.9923,0.6713,0.0154
4,BXD1,m,001-0525,CD,24.27,24.62,25.54,25.68,26.55,26.92,...,NaN,NaN,0.4231,0.2954,0.1772,0.5190,1.1283,0.2938,0.2840,0.0129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,BXD73b,m,103-0887,CD,31.74,33.19,37.97,38.20,38.84,40.10,...,44.2,10.00,1.3150,0.2837,0.2202,0.5235,1.3588,1.5501,1.6231,0.0174
512,BXD73b,m,103-0888,CD,31.32,33.47,31.28,37.53,39.17,40.61,...,42.0,9.60,1.0718,0.2705,0.2169,0.6021,1.2071,0.8027,1.1924,0.0169
513,BXD73b,m,103-0891,HFD,29.95,35.27,39.66,42.24,43.45,45.63,...,46.0,10.24,1.4626,0.3245,0.2115,0.5574,1.9336,1.5569,3.1686,0.0252
514,BXD73b,m,103-0892,HFD,28.81,31.45,35.22,38.55,41.33,44.07,...,44.2,10.20,1.5649,0.3048,0.2152,0.5865,1.9029,1.7150,2.4550,0.0221


In [162]:
pheno.columns

Index(['strain', 'sex', 'id', 'diet', 'bw_8wks', 'bw_9wks', 'bw_10wks',
       'bw_11wks', 'bw_12wks', 'bw_13wks',
       ...
       'hct', 'rbc', 'epid_fat_wt', 'muscle_wt', 'heart_wt', 'kidney_wt',
       'liver_wt', 'perirenal_wt', 'subcut_adip_wt', 'soleus_wt'],
      dtype='object', length=121)

In [163]:
pheno['bw_change'] = pheno['bw_presac'] - pheno['bw_8wks']
pheno['bw_change']

0       7.24
1       5.63
2       4.81
3      12.46
4       7.64
       ...  
511    11.60
512     8.31
513    23.74
514    23.92
515    21.09
Name: bw_change, Length: 460, dtype: float64

In [164]:
pheno_CD = pheno.loc[pheno["diet"]=="CD"]
pheno_HFD = pheno.loc[pheno["diet"]=="HFD"]

In [165]:
def strainAvgs(dataset, groupCol, var, newColAvgName):
    dataset_new = pd.DataFrame(dataset.groupby(groupCol, sort=False)[var].mean())
    dataset_new = dataset_new.rename({var: newColAvgName}, axis=1)  # new method
    dataset_new.insert(0, groupCol, dataset_new.index)
    dataset_new = dataset_new.reset_index(drop=True)
    return dataset_new


In [166]:
avg_change_CD = strainAvgs(pheno_CD, 'strain', 'bw_change', 'bw_change_avg_CD')
avg_change_HFD = strainAvgs(pheno_HFD, 'strain', 'bw_change','bw_change_avg_HFD')

In [167]:
avg_change_CD

,strain,bw_change_avg_CD
0,BXD1,7.556000
1,BXD2,8.806667
2,BXD6,11.086000
3,BXD8,7.185000
4,BXD11,11.850000
5,BXD12,6.146000
6,BXD16,18.902500
7,BXD27,7.066000
8,BXD32,7.498000
9,BXD34,11.775000


In [110]:
pheno_HFD =pd.merge(pheno_HFD, avg_change_HFD, on='strain', how='outer')
pheno_HFD =pd.merge(pheno_HFD, avg_change_CD, on='strain', how='outer')

In [111]:
pheno_HFD['bw_change_ratio'] = pheno_HFD['bw_change']/pheno_HFD['bw_change_avg_CD']

In [118]:
pheno_HFD = pheno_HFD[pheno_HFD['bw_change_ratio'].notna()]

In [119]:
pheno_HFD

,strain,sex,id,diet,bw_8wks,bw_9wks,bw_10wks,bw_11wks,bw_12wks,bw_13wks,...,heart_wt,kidney_wt,liver_wt,perirenal_wt,subcut_adip_wt,soleus_wt,bw_change,bw_change_avg_HFD,bw_change_avg_CD,bw_change_ratio
0,BXD1,m,001-0526,HFD,24.77,27.45,29.10,30.60,31.40,32.84,...,0.1744,0.4650,1.1210,0.8930,1.4620,0.0176,20.10,16.332,7.556,2.660138
1,BXD1,m,001-0527,HFD,24.48,26.88,27.53,27.89,27.91,28.68,...,0.1603,0.5080,1.0710,1.1110,1.1346,0.0175,15.01,16.332,7.556,1.986501
2,BXD1,m,001-0528,HFD,24.92,27.18,28.84,32.10,34.73,37.34,...,0.1299,0.4080,0.8197,0.6380,0.9112,0.0156,8.24,16.332,7.556,1.090524
3,BXD1,m,001-0529,HFD,23.88,26.16,28.60,31.00,32.13,33.54,...,0.1411,0.4670,1.0336,1.4770,1.8230,0.0176,18.92,16.332,7.556,2.503970
4,BXD1,m,001-0530,HFD,23.84,26.60,27.92,29.08,30.77,31.80,...,0.1767,0.4420,1.0954,1.2990,1.4248,0.0168,19.39,16.332,7.556,2.566173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,BXD73b,m,103-0885,HFD,30.19,34.98,38.48,41.77,44.02,46.47,...,0.2071,0.5241,1.5179,1.6805,2.9680,0.0210,21.80,22.442,10.280,2.120623
222,BXD73b,m,103-0886,HFD,27.06,31.25,33.98,36.99,39.56,42.26,...,0.2127,0.4641,1.4453,1.7368,2.0280,0.0204,21.66,22.442,10.280,2.107004
223,BXD73b,m,103-0891,HFD,29.95,35.27,39.66,42.24,43.45,45.63,...,0.2115,0.5574,1.9336,1.5569,3.1686,0.0252,23.74,22.442,10.280,2.309339
224,BXD73b,m,103-0892,HFD,28.81,31.45,35.22,38.55,41.33,44.07,...,0.2152,0.5865,1.9029,1.7150,2.4550,0.0221,23.92,22.442,10.280,2.326848


In [6]:
geno = pd.read_csv("Fixed_all_geno.csv", delimiter=",", index_col=0)
geno

,colGeno,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col421360,col421361,col421362,col421363,col421364,col421365,col421366,col421367,col421368,col421369
0,BXD1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BXD2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BXD6,2,2,0,2,0,2,2,0,2,...,0,0,0,0,0,0,0,0,0,0
3,BXD8,2,2,0,2,0,2,2,0,2,...,0,0,0,0,0,0,0,0,0,0
4,BXD11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,BXD12,2,2,0,2,0,2,2,0,2,...,0,0,0,0,0,0,0,0,0,0
6,BXD16,2,2,0,2,0,2,2,0,2,...,0,0,0,0,0,0,0,0,0,0
7,BXD27,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,BXD32,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,BXD34,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#colNames = geno.columns
strain_count = pheno.strain.value_counts(sort = False)
geno2=pd.DataFrame(np.repeat(geno.values, strain_count, axis=0))
geno2.columns = geno.columns
geno2

,colGeno,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col421360,col421361,col421362,col421363,col421364,col421365,col421366,col421367,col421368,col421369
0,BXD1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BXD1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BXD1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BXD1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BXD1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,BXD73b,2,2,0,2,0,2,2,0,2,...,0,0,0,0,0,0,0,0,0,0
512,BXD73b,2,2,0,2,0,2,2,0,2,...,0,0,0,0,0,0,0,0,0,0
513,BXD73b,2,2,0,2,0,2,2,0,2,...,0,0,0,0,0,0,0,0,0,0
514,BXD73b,2,2,0,2,0,2,2,0,2,...,0,0,0,0,0,0,0,0,0,0


## ANOVA TEST

In [120]:
f_oneway(*((pheno_HFD[pheno_HFD['strain'] == s]['bw_change_ratio']) for s in pheno_HFD['strain'].unique()))

F_onewayResult(statistic=6.472297687091711, pvalue=6.762816028332028e-21)

### This result helps show that there's a definite difference in the mean ratio for change in bw per strain. 

## MODEL TIME WAHOO 

In [168]:
y_CD = pd.DataFrame(pheno_CD.groupby('strain', sort=False)['bw_change'].mean())
CD_strains = list(y_CD.index.values)
X_CD = geno.loc[geno['colGeno'].isin(CD_strains)]
X_CD = X_CD.set_index('colGeno')

In [157]:
ridge_CD = RidgeCV(store_cv_values=True)
ridge_CD.fit(X_CD, y_CD)
print(ridge_CD.coef_)
print(ridge_CD.alpha_)
print("R^2: ", ridge_CD.score(X_CD, y_CD))
print("MSE: ", ridge_CD.cv_values_)

[[0.00055633 0.00043911 0.00036156 ... 0.         0.         0.        ]]
10.0
R^2:  0.9999999439591395
MSE:  [[[8.72986070e-01 8.72884468e-01 8.71868435e-01]]

 [[4.08570821e+00 4.08587479e+00 4.08753845e+00]]

 [[1.05215273e+01 1.05219874e+01 1.05265796e+01]]

 [[1.41760633e+00 1.41768223e+00 1.41844259e+00]]

 [[1.59466741e+00 1.59465757e+00 1.59455891e+00]]

 [[2.04478394e-01 2.04576498e-01 2.05558249e-01]]

 [[4.03586244e+01 4.03592058e+01 4.03650132e+01]]

 [[3.36349018e+00 3.36350112e+00 3.36361090e+00]]

 [[5.06538664e+00 5.06549844e+00 5.06661795e+00]]

 [[6.92603726e-01 6.92679689e-01 6.93438555e-01]]

 [[1.21280749e+01 1.21285186e+01 1.21329561e+01]]

 [[5.14030077e+01 5.14020460e+01 5.13924387e+01]]

 [[8.71919384e+00 8.71877441e+00 8.71458147e+00]]

 [[2.28876964e+00 2.28874958e+00 2.28854880e+00]]

 [[4.23684309e-01 4.23592989e-01 4.22681179e-01]]

 [[1.69220818e+01 1.69225190e+01 1.69268846e+01]]

 [[2.91416405e+01 2.91404118e+01 2.91281386e+01]]

 [[4.25116819e+01 4.251

In [144]:
y_HFD = pd.DataFrame(pheno_HFD.groupby('strain', sort=False)['bw_change'].mean())
HFD_strains = list(y_HFD.index.values)
X_HFD = geno.loc[geno['colGeno'].isin(HFD_strains)]
X_HFD = X_HFD.set_index('colGeno')


In [159]:
ridge_HFD = RidgeCV(store_cv_values=True)
ridge_HFD.fit(X_HFD, y_HFD)
print(ridge_HFD.coef_)
print(ridge_HFD.alpha_)
print("R^2: ", ridge_HFD.score(X_HFD, y_HFD))
print("MSE: ", ridge_HFD.cv_values_)

[[5.46062067e-04 3.30240933e-04 3.52166590e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
10.0
R^2:  0.999999955909098
MSE:  [[[1.11372498e+01 1.11385541e+01 1.11515925e+01]]

 [[5.48242508e+00 5.48184498e+00 5.47604815e+00]]

 [[4.98291921e+00 4.98271020e+00 4.98062220e+00]]

 [[5.95829717e+00 5.95704259e+00 5.94451153e+00]]

 [[6.42229139e+01 6.42236815e+01 6.42313508e+01]]

 [[6.41577836e+01 6.41593271e+01 6.41747564e+01]]

 [[7.34255580e+00 7.34234241e+00 7.34021524e+00]]

 [[1.00616300e+02 1.00617505e+02 1.00629551e+02]]

 [[7.81529436e-01 7.81754515e-01 7.84004448e-01]]

 [[2.57554038e+02 2.57554876e+02 2.57563228e+02]]

 [[3.10519226e+01 3.10514882e+01 3.10471480e+01]]

 [[1.83583820e+02 1.83578038e+02 1.83520276e+02]]

 [[1.11181027e+01 1.11185002e+01 1.11224740e+01]]

 [[9.40880600e+01 9.40895789e+01 9.41047681e+01]]

 [[3.34595076e+01 3.34578728e+01 3.34415443e+01]]

 [[1.48057879e-01 1.48105121e-01 1.48577015e-01]]

 [[4.47162521e+01 4.47156491e+01 4.47096168e+01]]

### Note on potential model adjustments: We might want to try adding weights to the model, since different strains had different numbeer of samples. We could also play around with alpha

### I think the MSE's above are technically the correct ones to use for LOOCV, but I'm leaving these MSEs below in just in case for now

In [152]:
y_CD_pred = ridge_CD.predict(X_CD)
y_CD_pred = np.array(y_CD_pred).T.tolist()[0]
y_CD_true = y_CD['bw_change'].values.tolist()

[7.555840173682928,
 8.807002253772424,
 11.086613004127585,
 7.185210307663598,
 11.849771513896467,
 6.146078345997876,
 18.901424906497923,
 7.066229456260878,
 7.49839908651147,
 11.774863681455551,
 17.077339715530698,
 4.304507851810575,
 11.417555572845455,
 13.953070363144997,
 9.689898661962609,
 11.626307859131977,
 9.418412114734124,
 7.606072737321425,
 11.477740421613023,
 11.21994640152192,
 7.602522963436561,
 7.483497758492703,
 17.399837928350333,
 8.53551739346161,
 9.41018770006306,
 10.22009274851904,
 21.00229268030468,
 21.23643340050405,
 11.740563314107089,
 16.376594116684057,
 12.461121587796736,
 7.977996134182018,
 12.677732950011855,
 7.379000101358268,
 8.212755708019634,
 8.270300290188711,
 14.549052965626819,
 15.827851879107211,
 7.437694409763832,
 7.4434282961167035,
 12.42655454988565,
 17.90096489080039,
 6.480942698586961,
 9.188035468558382,
 11.205312939115753,
 23.82008980979367,
 11.890503227288814,
 17.401536219926193,
 16.403505017459832,
 1

In [155]:
y_HFD_pred = ridge_HFD.predict(X_HFD)
y_HFD_pred = np.array(y_HFD_pred).T.tolist()[0]
y_HFD_true = y_HFD['bw_change'].values.tolist()

In [154]:
mean_squared_error(y_CD_true, y_CD_pred)

1.036588661440098e-06

In [156]:
mean_squared_error(y_HFD_true, y_HFD_pred)

2.1090817475498995e-06

### Very pog. Thank god. I was gonna to lose my mind just a little if the result was bad. Hopefully I did this correctly LOL

## OLD STUFF I WILL DELETE LATER

In [7]:
'''def getColNames(length):
    names = []
    for i in range(length):
        name = "SNP" + str(i+1)
        #print(name)
        names.append(name)
    return names
'''

In [8]:
'''geno = pd.read_table("animal/mouse.geno", delimiter=",", header=None)
geno.columns = ["strain","SNPString"]
dfSNPs = geno.SNPString.apply(lambda x: pd.Series(list(x)))
snpColNames = getColNames(len(dfSNPs.columns))
dfSNPs.columns = snpColNames
geno = geno.drop(["SNPString"], axis=1)
geno = geno.join(dfSNPs)
colNames = geno.columns
strain_count = pheno.strain.value_counts(sort = False)
geno=pd.DataFrame(np.repeat(geno.values, strain_count, axis=0))
geno.columns = colNames
geno'''

,strain,SNP1,SNP2,SNP3,SNP4,SNP5,SNP6,SNP7,SNP8,SNP9,...,SNP421360,SNP421361,SNP421362,SNP421363,SNP421364,SNP421365,SNP421366,SNP421367,SNP421368,SNP421369
0,BXD1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BXD1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BXD1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BXD1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BXD1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,BXD73b,2,2,0,2,0,2,2,0,2,...,0,0,0,0,0,0,0,0,0,0
512,BXD73b,2,2,0,2,0,2,2,0,2,...,0,0,0,0,0,0,0,0,0,0
513,BXD73b,2,2,0,2,0,2,2,0,2,...,0,0,0,0,0,0,0,0,0,0
514,BXD73b,2,2,0,2,0,2,2,0,2,...,0,0,0,0,0,0,0,0,0,0
